In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import random
from datetime import datetime
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#URL 정의
#data/Mylist
MYLIST_URL = '/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Mylist.csv'

#data/Base_song_meta
SONG_META_URL = '/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Base_Song_meta.csv'

#data/Play_list
PLAYLIST_URL = '/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Play_list.csv'

day = datetime.today().strftime('%Y%m%d')
#/일간멜론DJ/
DAILY_PLAYLIST_URL = '/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간멜론DJ/'+day+'일간멜론DJ.csv'

#/일간랭킹
day = str(int(day)-2)
DAILY_RANK_URL = '/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간랭킹/'+day+'일간랭킹_모든항목.csv'

# 1. 검색할 태그 추가
> 1. 태그를 데이터에 추가하여 인덱스를 가지고 있는다.
2. 유사도 분석후 가지고있는 인덱스로 플레이리스트를 검색한다.

##list로 바꿔주는 부분

In [ ]:
#태그리스트 만들때 사용할 split함수
def StrtoList(item):
  return item.split(' ')
#장르리스트
def GenretoList(item):
  return item.split(', ')
#아티스트리스트
def ArtisttoList(item):
  return item.split(' , ')

##오늘의 추천태그 부분

In [ ]:
def getMyTagTop10():
  mylist = pd.read_csv(MYLIST_URL)
  mylist['index'] = mylist['번호']
  mylist = mylist.set_index('index')
  mylist = mylist.iloc[0:, 1:]    #마이리스트 전처리

  mylist['태그'] = mylist['태그'].astype(str)   
  mylist['태그_리스트'] = mylist['태그'].apply(StrtoList)
  mylist = mylist.fillna('')

  tag_list = []     #top_10태그 뽑는부분
  for idx in mylist.index:
    tag_list += mylist.loc[idx]['태그_리스트']
  tag_list_df = pd.DataFrame(tag_list)
  tag_top_10 = []
  for i in range(1, 11):
    tag_top_10.append(tag_list_df.value_counts().index[i][0])
  return (' ').join(random.sample(tag_top_10, k=10))

def getPlyTagTop10():
  daily_df = pd.read_csv(DAILY_PLAYLIST_URL)
  daily_df['태그리스트'] = daily_df['플레이리스트_태그'].apply(StrtoList)
  tag_list = []
  for idx in range(len(daily_df.index)):
    tag_list += daily_df.iloc[idx]['태그리스트']

  tag_df = pd.DataFrame(tag_list)
  tag_sorted=[]
  for item in tag_df.value_counts().index:
    tag_sorted.append(item[0])

  return (' ').join(tag_sorted[:10])

## 날씨태그 부분

In [ ]:
def getWeatherTag():
  spring = ['03','04','05'] ; summer = ['06','07','08'] ; fall = ['09','10','11'] ; winter = ['12','01','02']

  spring_tags = '봄,벚꽃,봄캐롤,개강,봄소풍'.split(',')
  summer_tags = '여름,Summer,여름밤,여름노래,열대야,초여름,트로피칼사운드,트로피컬,바캉스'.split(',')
  fall_tags = '가을,개강,가을_공통,쌀쌀한'.split(',')
  winter_tags = '겨울,크리스마스,눈'.split(',')
  sunny_tags = '청량,청량함,상쾌한,청량한,시원한'.split(',')
  rain_tags = '비오는날,비,우산,쌀쌀한'.split(',')
  snow_tags = '추운날,쌀쌀한,눈'.split(',')
  # key 번호
  my_key = 'Zpnys12cNaw5Me7zHDcC14kUKxMbJPLaq2IK8B5MHkgVz8Q66zqa3KRkyskVCQFRs9pYBgXNlegXyuViozZitA%3D%3D'

  # 날씨 api 디코딩
  import datetime
  end_point1 = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/'
  end_point2 = 'getUltraSrtNcst?'
  parm_key = 'serviceKey=' + my_key

  current = datetime.datetime.now()
  today = current + datetime.timedelta(hours=8)
  base_time = today.strftime("%H%M")
  base_date = today.strftime("%Y%m%d")
  base_month = today.strftime("%m")

  rows = '10' ; page = '1' ; nx = '60' ; ny = '127' # 위치 : 서울특별시 중구 동아빌딩
  parm_data = '&dataType=JSON' ; parm_rows = '&numOfRows=' + rows ; parm_page = '&pageNo=' + page ; parm_date = '&base_date=' + base_date ; parm_time = '&base_time=' + base_time ; parm_nx = '&nx=' + nx ; parm_ny = '&ny=' + ny
  url = end_point1 + end_point2 + parm_key + parm_data + parm_rows + parm_page  + parm_date + parm_time + parm_nx + parm_ny
  # print(url)

  # 날씨 코드 분류
  import requests
  res1 = requests.get(url)
  items = res1.json().get('response').get('body').get('items')

  data = dict()
  data['data'] = base_date

  weather_data = dict()
  for item in items['item']:
    # 기온
    if item['category'] == 'T1H':
      weather_data['tmp'] = item['obsrValue']
    # 습도
    if item['category'] == 'REH':
      weather_data['hum'] = item['obsrValue']
    # 기상상태
    if item['category'] == 'PTY':
      weather_code = item['obsrValue']
      if weather_code == '1':
        weather_state = '비'
      elif weather_code == '2':
        weather_state = '비/눈'
      elif weather_code == '3':
        weather_state = '눈'
      elif weather_code == '5':
        weather_state = '빗방울'
      elif weather_code == '6':
        weather_state = '빗방울눈날림'
      elif weather_code == '7':
        weather_state = '눈날림'
      else:
        weather_state = '없음'

      weather_data['code'] = weather_code
      weather_data['state'] = weather_state

  weather_data['month'] = base_month
  data['weather'] = weather_data
  # print(data['weather'])

  # 날씨별 상태 분류
  if base_month == base_month:
    if data.get('weather').get('code') == '0': # 맑음
      weather_state = '1'
    elif data.get('weather').get('code') == '1' or '5': # 비
      weather_state = '2'
    elif data.get('weather').get('code') == '3' or '7': # 눈
      weather_state = '3'
    elif data.get('weather').get('code') == '2' or '6': # 비/눈
      weather_state = '4'
    else:
      weather_state = ''
  else:
    weather_state = 'season error!'

  # 날씨 상태별 태그 추출
  import random
  if base_month in spring:
    if weather_state == '1':
      weather_list = random.sample(spring_tags + sunny_tags, k=len(spring_tags + sunny_tags))
    elif weather_state == '2':
      weather_list = random.sample(spring_tags + rain_tags, k=len(spring_tags + rain_tags))
    elif weather_state == '3':
      weather_list = random.sample(spring_tags + snow_tags, k=len(spring_tags + snow_tags))
    else:
      weather_list = random.sample(spring_tags + rain_tags + snow_tags, k=len(spring_tags + rain_tags + snow_tags))
  elif base_month in summer:
    if weather_state == '1':
      weather_list = random.sample(summer_tags + sunny_tags, k=len(summer_tags + sunny_tags))
    elif weather_state == '2':
      weather_list = random.sample(summer_tags + rain_tags, k=len(summer_tags + rain_tags))
    elif weather_state == '3':
      weather_list = random.sample(summer_tags + snow_tags, k=len(summer_tags + snow_tags))
    else:
      weather_list = random.sample(summer_tags + rain_tags + snow_tags, k=len(summer_tags + rain_tags + snow_tags))
  elif base_month in fall:
    if weather_state == '1':
      weather_list = random.sample(fall_tags + sunny_tags, k=len(fall_tags + sunny_tags))
    elif weather_state == '2':
      weather_list = random.sample(fall_tags + rain_tags, k=len(fall_tags + rain_tags))
    elif weather_state == '3':
      weather_list = random.sample(fall_tags + snow_tags, k=len(fall_tags + snow_tags))
    else:
      weather_list = random.sample(fall_tags + rain_tags + snow_tags, k=len(fall_tags + rain_tags + snow_tags))
  else:
    if weather_state == '1':
      weather_list = random.sample(winter_tags + sunny_tags, k=len(winter_tags + sunny_tags))
    elif weather_state == '2':
      weather_list = random.sample(winter_tags + rain_tags, k=len(winter_tags + rain_tags))
    elif weather_state == '3':
      weather_list = random.sample(winter_tags + snow_tags, k=len(winter_tags + snow_tags))
    else:
      weather_list = random.sample(winter_tags + rain_tags + snow_tags, k=len(winter_tags + rain_tags + snow_tags))

  #날씨 태그중 5개를 랜덤으로 선택해서 반환해준다.
  weather_list = random.sample(weather_list, k=5)
  for idx in range(len(weather_list)):
    if weather_list[idx][0] != '#':
      weather_list[idx] = '#' + weather_list[idx]
  print(weather_list)
  return weather_list

## 플레이리스트 기반 부분

In [ ]:
#플레이리스트 기반 추천으로 사용할 플레이리스트 정보 뽑은 함수
def PlaylistBase():
  #플레이리스트에서 태그를 뽑아 top10을 만들어줌
  mylist = pd.read_csv(MYLIST_URL)
  mylist['index'] = mylist['번호']
  mylist = mylist.set_index('index')
  mylist = mylist.iloc[0:, 1:]    #마이리스트 전처리

  mylist['태그'] = mylist['태그'].astype(str)   
  mylist['태그_리스트'] = mylist['태그'].apply(StrtoList)
  mylist = mylist.fillna('')

  tag_list = []     #top_10태그 뽑는부분
  for idx in mylist.index:
    tag_list += mylist.loc[idx]['태그_리스트']
  tag_list_df = pd.DataFrame(tag_list)
  tag_top_10 = []
  for i in range(1, 11):
    tag_top_10.append(tag_list_df.value_counts().index[i][0])
  tags = (' ').join(random.sample(tag_top_10, k=5))


  #가수명과, 가수번호 뽑기
  mylist['가수'] = mylist['가수'].astype(str)
  mylist['가수번호'] = mylist['가수번호'].astype(str)

  art_list = (' , ').join(mylist['가수'].values.tolist()).split(' , ')
  art_num_list = (' ').join(mylist['가수번호'].values.tolist()).split(' ')
  artist_tup = tuple(zip(art_list,art_num_list))
  artist_set = set(artist_tup)
  if ('nan', 'nan') in artist_set:
    artist_set.remove(('nan', 'nan'))
  art_list = []
  art_num = []

  for item in artist_set:
    art_list.append(item[0])
    art_num.append(item[1])

  artist = (' , ').join(art_list)
  artist_num = (' ').join(art_num)
  
  #장르 top10뽑기
  mylist['장르'] = mylist['장르'].astype(str)
  genre_list = []
  for item in mylist['장르'].apply(GenretoList).values.tolist():
    genre_list += item
  genre_index = pd.DataFrame(genre_list).value_counts().index

  genre_top_10 = []
  for item in genre_index:
    genre_top_10.append(item[0])

  genre = (', ').join(genre_top_10[:5])

  return tags, artist, artist_num, genre

## 플레이리스트 top3장르로 인기순위에서 20개씩 뽑는함수

In [ ]:
def getDailyTop():
  mylist = pd.read_csv(MYLIST_URL)
  mylist['index'] = mylist['번호']
  mylist = mylist.set_index('index')
  mylist = mylist.iloc[0:, 1:]    #마이리스트 전처리

  daily_df = pd.read_csv(DAILY_RANK_URL)

  #장르 top10뽑기
  mylist['장르'] = mylist['장르'].astype(str)
  genre_list = []
  for item in mylist['장르'].apply(GenretoList).values.tolist():
    genre_list += item
  genre_index = pd.DataFrame(genre_list).value_counts().index

  genre_top_10 = []
  for item in genre_index:
    genre_top_10.append(item[0])

  genre_top_10 = genre_top_10[:10]

  daily_genre_top = pd.DataFrame()
  for item in genre_top_10:
    daily_genre_top = pd.concat([daily_genre_top, daily_df[daily_df['장르']==item].head(20)])
    if len(daily_genre_top) >= 60:
      break
  return daily_genre_top

##태그로 플레이리스트 추천받는 부분

In [ ]:
def TagSearch(user_tag=''):
  if user_tag=='':
    user_tag = getWeatherTag()

  print('입력된태그', user_tag)
  ply_list_meta = pd.read_csv(PLAYLIST_URL, index_col=0)
  ply_list_meta['태그_리스트'] = ply_list_meta['플레이리스트_태그'].apply(StrtoList)

  tag_list = []
  for item in user_tag:
    for idx in range(len(ply_list_meta.index)):
      if item in ply_list_meta.iloc[idx]['태그_리스트']:
        tag_list+=ply_list_meta.iloc[idx]['태그_리스트']
        tag_list.pop(tag_list.index(item))
  tag_list
  tag_df = pd.DataFrame(tag_list)
  tag_list = []
  for item in tag_df.value_counts().index:
    tag_list.append(item[0])

  tag_top_10 = tag_list[:10]

  tag_top_5 = random.sample(tag_top_10, k=5)

  tag = user_tag + tag_top_5
  return tag

##가중치 부분

In [ ]:
#가중치 계산하는 함수
def addPoint(data_frame, sim_list, new_index):
  df = data_frame.loc[sim_list]
  df['가수번호리스트'] = df['가수번호'].apply(StrtoList)
  df['장르리스트'] = df['장르'].apply(GenretoList)
  df['점수'] = 0
  art_num = df.loc[new_index]['가수번호리스트']
  genre_list = df.loc[new_index]['장르']
  test= []
  for idx in range(1, len(df.index)):
    for item in df['가수번호리스트'].iloc[idx]:
      if item in art_num:
        df['점수'].iloc[idx] += 1
    for item in df['장르리스트'].iloc[idx]:
      if item in genre_list:
        df['점수'].iloc[idx] += 1
  
  return df.sort_values('점수', ascending=False)


##함수 메인부분

In [ ]:
#검색할 태그를 조정하려면 여기를 조정하면 될 것이다.
#또한 for문을 돌리더라도 여기서부터 시작해야한다.
def getTags(flag, artist = '', artist_num = '', genre = '', tag=''):
  if flag == 0:
    #테스트 태그
    tag =  (' ').join(['#발라드', '#이별', '#슬픔'])

  elif flag == 1:
    print(flag)
    #날씨
    tag = getWeatherTag()

  elif flag == 2:
    print(flag)
    #플레이리스트 추천
    tag, artist, artist_num, genre = PlaylistBase()

  elif flag == 3:
    print(flag)
    #플레이리스트 기반 많이나온 장르 3가지로 인기순위에서 20개씩 뽑아서 추천
    return getDailyTop()

  elif flag == 4:
    print(flag)
    #태그로 추천
    #태그를 입력하면 리스트로 만들어줌
    item = input('태그를 입력해주세요! : ')
    input_tag = []
    while (item != ' '):
      input_tag.append(item)
      item = input('태그를 입력해주세요! : ')
    for idx in range(len(input_tag)):
      if input_tag[idx][0] != '#':
        input_tag[idx] = '#' + input_tag[idx]
    try:
      tag = TagSearch(input_tag)
    except:
      print('태그가 존재하지 않습니다. 기본태그로 리스트를 생성합니다.')
      TagSearch('')
    print(tag)

  if func == 4: 
    return pd.DataFrame(['', '', '', '' ,user_tag], index=['플레이리스트_번호', '멜론DJ', '플레이리스트', '번호', '플레이리스트_태그']).transpose()
  else: 
    return pd.DataFrame(['', '',  '', artist, artist_num, '', genre, '' ,tag], index=['번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']).transpose()

#getTags(3)

# 2. 데이터 불러오기 이후 유사도 측정까지

In [ ]:
#유사도 측정
def getSimMatrix(data_frame, col):
  #단어 묶음 리스트화
  from sklearn.feature_extraction.text import CountVectorizer
  count_vect = CountVectorizer(ngram_range=(1,10)).fit(data_frame[col])

  tag_matrix = count_vect.fit_transform(data_frame[col])

  #유사도 분석
  from sklearn.metrics.pairwise import cosine_similarity
  tag_similarity = cosine_similarity(tag_matrix, tag_matrix)

  #유사도 매트릭스 내림차순으로 정렬
  tag_similarity_sorted_idx = tag_similarity.argsort()[:, ::-1]
  return tag_similarity_sorted_idx

#3. 메인

In [ ]:
#검색어 설정 부분
col = ['번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']
# func = (0: 테스트, 1: 날씨, 2:플레이리스트, 3:인기순위, 4:태그 검색)
text='''
명령어를 입력하세요.
0: 테스트
1: 날씨
2:플레이리스트
3:인기순위
4:태그 검색

오늘의 인기태그
'''

text += getPlyTagTop10()+'\n\n오늘의 추천태그\n' + getMyTagTop10()+'\n'
func = int(input(text))
#func = 4

if func == 3:
  result = getTags(func)

else:
  new_tag = getTags(func)

  #데이터 불러오기
  if func == 4:
    ply_list_meta = pd.read_csv(PLAYLIST_URL, index_col=0)
    new_tag_df = pd.concat([ply_list_meta, new_tag], axis=0, ignore_index=True)
    new_index = len(new_tag_df.index)-1

    #결측치 지우기
    new_tag_df = new_tag_df.fillna('')

    #태그 문자열화 -> 태그 리스트화
    new_tag_df['플레이리스트_태그'] = new_tag_df['플레이리스트_태그'].astype(str)
    new_tag_df['플레이리스트_태그_리스트'] = new_tag_df['플레이리스트_태그'].apply(StrtoList)

    #유사도 리스트 가져오기
    tag_similarity_sorted_idx = getSimMatrix(new_tag_df, '플레이리스트_태그')

  else:
    song_meta = pd.read_csv(SONG_META_URL, index_col=0)
    song_meta = song_meta.iloc[0:, 1:]

    new_tag_df = pd.concat([song_meta, new_tag], axis=0, ignore_index=True)
    new_index = len(new_tag_df.index)-1
    #결측치 지우기
    new_tag_df = new_tag_df.fillna('')

    #태그 문자열화 -> 태그 리스트화
    new_tag_df['태그'] = new_tag_df['태그'].astype(str)
    new_tag_df['태그_리스트'] = new_tag_df['태그'].apply(StrtoList)

    #유사도 리스트 가져오기
    tag_similarity_sorted_idx = getSimMatrix(new_tag_df, '태그')

  #func이 2일때(기존 플레이리스트) 가중치 넣기
  if func == 2:
    sorted_df = addPoint(new_tag_df.copy(), tag_similarity_sorted_idx[new_index], new_index)
    sim_list = sorted_df.index
  else:
    sim_list = tag_similarity_sorted_idx[new_index]

  #top20만 뽑아 검색하기
  top_20 = list(sim_list[:20])
  if new_index in top_20:
    top_20 = list(sim_list[:21])
    top_20.pop(top_20.index(new_index))
  result = new_tag_df.loc[top_20]
result


명령어를 입력하세요.
0: 테스트
1: 날씨
2:플레이리스트
3:인기순위
4:태그 검색

오늘의 인기태그
#기분전환 #인디 #인디밴드 #드라이브 #명곡 #록 #국내록 #Rock #신예밴드 #국내밴드

오늘의 추천태그
#겨울 #잔잔한 #휴식 #새벽 #사랑 #추억 #까페 #OST #발라드 #밤
1
1
['#청량함', '#상쾌한', '#청량한', '#청량', '#바캉스']


,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그,태그_리스트
516,4394552,2226060,Calls & Echoes,Melissa Polinar,513429,Happy,포크/블루스/컨트리,"2,331",#팝 #아침 #상쾌한,"[#팝, #아침, #상쾌한]"
2615,32225770,10359552,Memories (feat. Ben Woodward),Zita,2736199,Memories (feat. Ben Woodward),POP,975,#감성 #청량함 #밤 #여름밤,"[#감성, #청량함, #밤, #여름밤]"
1577,32517163,10414491,Please Don't Go,Mykey,2638205,Please Don't Go,포크/블루스/컨트리,322,#여름밤 #팝 #장르불문 #청량함,"[#여름밤, #팝, #장르불문, #청량함]"
2375,3957658,2166545,Take Me Home (Standard Edition),One Direction,581612,Summer Love,POP,"2,140",#감성 #청량함 #여름밤 #팝,"[#감성, #청량함, #여름밤, #팝]"
493,4185315,2194331,Dot To Dot,Tim Myers,402680,Simply Wonderful,포크/블루스/컨트리,232,#팝 #카페 #아침 #상쾌한,"[#팝, #카페, #아침, #상쾌한]"
486,4137990,2188360,Morning Sunshine,Julien May,725847,Badly Hooked On You,포크/블루스/컨트리,"6,615",#팝 #카페 #아침 #상쾌한,"[#팝, #카페, #아침, #상쾌한]"
2391,4646588,2256290,꽃갈피,아이유,261143,꿍따리 샤바라 (Feat. 클론),발라드,"30,298",#여름 #바캉스 #기분전환,"[#여름, #바캉스, #기분전환]"
2655,32574989,10425995,this time last year,Nightly,1134197,this time last year,"POP, 록/메탈",269,#청량함 #버스 #여름밤 #팝,"[#청량함, #버스, #여름밤, #팝]"
418,3639165,2046806,Forever And Ever,David Choi,330771,Happily Ever After,포크/블루스/컨트리,"11,974",#팝 #카페 #아침 #상쾌한,"[#팝, #카페, #아침, #상쾌한]"
417,3639155,2046806,Forever And Ever,David Choi,330771,This And That Is Life (SSG 쓱 광고배경음악),포크/블루스/컨트리,"2,771",#팝 #카페 #아침 #상쾌한,"[#팝, #카페, #아침, #상쾌한]"


#테스트

In [ ]:
text='''
명령어를 입력하세요.
0: 테스트
1: 날씨
2:플레이리스트
3:인기순위
4:태그 검색
5:태그 추천

오늘의 인기태그
'''

text += getPlyTagTop10()+'\n\n오늘의 추천태그\n' + getMyTagTop10()+'\n'
#func = input(text)
#func
print(text)


명령어를 입력하세요.
0: 테스트
1: 날씨
2:플레이리스트
3:인기순위
4:태그 검색
5:태그 추천

오늘의 인기태그
#기분전환 #인디 #인디밴드 #드라이브 #명곡 #록 #국내록 #Rock #신예밴드 #국내밴드

오늘의 추천태그
#새벽 #휴식 #까페 #발라드 #사랑 #추억 #겨울 #밤 #OST #잔잔한

